In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from pylab import rcParams
rcParams['figure.figsize'] = 12,8

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns


import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import re

import os
from glob import glob
from tqdm import tqdm

import yaml
from yaml import dump
import uuid
import itertools

In [3]:
import xgboost
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from lightgbm import LGBMClassifier

In [4]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [5]:
import warnings
warnings.simplefilter(action="ignore")

## Initialization

In [6]:
# companies = ["AMAZON", "APPLE", "GOOGLE", "META", "NETFLIX"]
# time_period = ["daily", "weekly", "monthly"]

# path = f"/diploma_info/datalake/raw_data/{companies[2]}_{time_period[2]}.csv"

In [7]:
# date_parse = lambda dates: pd.to_datetime(dates)

# df = pd.read_csv(
#     path,
#     parse_dates=["Date"],
#     date_parser=date_parse,
#     index_col=["Date"],
# )

In [8]:
# growth = [0]
# diff_value = [0]


# for k in range(1, df.shape[0]):
#     diff_value.append(df.iloc[k]["Close"] - df.iloc[k-1]["Close"])
#     if diff_value[-1] > 0:
#         growth.append(1)
#     else:
#         growth.append(0)

In [9]:
# df['diff_value'] = diff_value
# df['growth'] = growth

## EDA

In [10]:
# df.shape[0]-df.growth.sum(), df.growth.sum()

In [11]:
# start = datetime(2023, 12, 1, 0)
# end = datetime(2024, 1, 31, 23)


# fig = make_subplots(specs=[[{"secondary_y": True}]])
# text = {"date": df.index.date,
#         "open": df.Open,
#         "high": df.High,
#         "low": df.Low,
#         "close": df.Close,
#        }


# fig.add_trace(go.Scatter(x=df.index.date,
#                          y=df.Close,
#                          name='Google',
#                          text=round(df.Open, 5),
#                          line_color='#000000',
#                          hovertemplate="Date: %{x}<br>Open: %{text}<br>Close: %{y}"
#                         ),
#                secondary_y=False
#              )

# colors = ['red' if df.iloc[k].growth == 0 else 'green' for k in range(df.shape[0])]

# fig.add_trace(go.Bar(x=df.index.date,
#                      y=df.diff_value,
#                      name='Growth_or_Fall',
#                      marker_color=colors,
#                      ),
#                secondary_y=True
#              )


# fig.update_layout(title=f'Electricity_consumption', 
#                   xaxis_tickangle=-45,
#                   width=1500,
#                   height=800
#                  )

# fig.update_layout(hovermode="x")
# fig.update_xaxes(rangeslider_visible=True)
# fig.update_yaxes(title_text="Close price value", secondary_y=False)
# fig.update_yaxes(title_text="Change", secondary_y=True)


# # fig.write_html()

# fig.show()

## Forecasting

#### initialize all required valiables, prepare datasets

In [12]:
def init(company, period):

    train_end = datetime(2023, 12, 31)
    test_start = datetime(2024, 1, 1)
    test_end = datetime(2024, 2, 8)
    
    train_features_set = [
        ['open', 'volume_lag_1', 'usa_inflation_%'],
        ['open', 'volume_lag_1', 'diff_close_value'],
        ['open', 'volume_lag_1', 'growth_open'],
#         ['open', 'diff_open_value', 'open-prev_close', 'year', 'month', 'day', 'day_of_week', 'week_of_year']
#         ['open', 'close_max_3_days', 'close_min_3_days', 'close_mean_3_days'],
#         ['open', 'year', 'month', 'day', 'day_of_week', 'week_of_year']
    ]

    
    date_parse = lambda dates: pd.to_datetime(dates)
    path = f"/diploma_info/datalake/processed_data/{company}_{period}.csv"
    
    full_set = pd.read_csv(
        path,
        parse_dates=["date"],
        date_parser=date_parse,
        index_col=["date"],
    )
#     full_set.index.name = 'date'
#     full_set.columns = [c.lower() for c in full_set.columns]
    
#     full_set = create_target_value(full_set)
#     full_set = create_new_features(full_set, test_end)

#     full_set['growth_lag_1', 'growth_lag_2', 'growth_lag_3'] = full_set

    full_set = full_set.fillna(0)
    
    # create _mixed values
#     full_set['close_mixed'] = full_set['close']
#     full_set.loc[test_start:, 'close_mixed'] = full_set.loc[test_start:, 'close_lag_1']
#     full_set['volume_mixed'] = full_set['volume']
#     full_set.loc[test_start:, 'volume_mixed'] = full_set.loc[test_start:, 'volume_lag_1']
    
    
    test_start = datetime(full_set.loc[test_start:].index[0].year, full_set.loc[test_start:].index[0].month, full_set.loc[test_start:].index[0].day)
    train_end = datetime(full_set.loc[:train_end].index[-1].year, full_set.loc[:train_end].index[-1].month, full_set.loc[:train_end].index[-1].day)
    
    
    return full_set, train_end, test_start, test_end, train_features_set

In [13]:
def create_target_value(df):

    growth = [0]
    diff_value = [0]


    for k in range(1, df.shape[0]):
        diff_value.append(df.iloc[k]["close"] - df.iloc[k-1]["close"])
        if diff_value[-1] > 0:
            growth.append(1)
        else:
            growth.append(0)

    df['diff_value'] = diff_value
    df['growth'] = growth
    
    return df

In [14]:
def create_new_features(df, test_end):
    
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['day_of_week'] = df.index.weekday
    df['week_of_year'] = (df.index.isocalendar()['week']).astype('int')
    df['close_lag_1'] = df['close'].shift(1).bfill()
    df['volume_lag_1'] = df['volume'].shift(1).bfill()

    for window in [3, 5, 7]:
        close_agg = pd.DataFrame(round(df['close'].rolling(window=window, closed='left').agg(
            ('max', 'min', 'mean')
        )))
        close_agg.columns = [f'close_max_{window}_days', f'close_min_{window}_days', f'close_mean_{window}_days']
        day_mean = close_agg.reset_index()[['date', f'close_max_{window}_days', f'close_min_{window}_days', f'close_mean_{window}_days']]

        df = df.reset_index().merge(day_mean, on='date').set_index("date")
        df = df.loc[:test_end.strftime("%Y%m%d"),]
    
    return df

In [15]:
def models_hyperparameter_random_forest(company):

    depth_list = []
    n_estimators_list = []
    
    if company == 'amazon':
        depth_list = [7]
        n_estimators_list = [5000]
    elif company == 'apple' or company == 'netflix':
        depth_list = [7]
        n_estimators_list = [10]
    elif company == 'meta':
        depth_list = [9]
        n_estimators_list = [10]
    elif company == 'google':
        depth_list = [9]
        n_estimators_list = [10, 200]
    
    hyperparameters_for_model = []
    
    for depth, n_estimators in itertools.product(depth_list, n_estimators_list):
        hyperparameters_for_model.append({
                        'n_estimators': n_estimators,
                        'n_jobs': -1,
                        'random_state': 2,
                        'max_depth': depth,
            })

    return hyperparameters_for_model


def models_hyperparameter_xgboost(company):

    depth_list = []
    n_estimators_list = []
    
    if company == 'apple' or company == 'google':
        depth_list = [7, 9]
        n_estimators_list = [10, 200]
    elif company == 'amazon':
        depth_list = [7]
        n_estimators_list = [10]
    elif company == 'meta':
        depth_list = [7]
        n_estimators_list = [5000]
    elif company == 'netflix':
        depth_list = [9]
        n_estimators_list = [200]
    
    hyperparameters_for_model = []
    
    for depth, n_estimators in itertools.product(depth_list, n_estimators_list):
        hyperparameters_for_model.append({
                    'n_estimators': n_estimators,
                    'n_jobs': -1,
                    'max_depth': depth,
                    'eta': 0.3,
                    'booster': 'gbtree',
                    'objective': 'reg:squarederror',
                    'eval_metric': 'rmse',
                    'subsample': 1,
                    'colsample_bytree': 1,
                    'min_child_weight': 1,
                    'random_state': 2,
            })
        
    return hyperparameters_for_model


def models_hyperparameter_lgbm(company):

    depth_list = []
    n_estimators_list = []
    
    if company == 'apple' or company == 'meta':
        depth_list = [7, 9]
        n_estimators_list = [5000]
    elif company == 'amazon':
        depth_list = [7]
        n_estimators_list = [200]
    elif company == 'google':
        depth_list = [7, 9]
        n_estimators_list = [10]
    elif company == 'netflix':
        depth_list = [9]
        n_estimators_list = [10, 5000]
    
    hyperparameters_for_model = []
    
    for depth, n_estimators in itertools.product(depth_list, n_estimators_list):
        hyperparameters_for_model.append({
                    'n_estimators': n_estimators,
                    'n_jobs': -1,
                    'max_depth': depth,
                    'eta': 0.3,
                    'random_state': 2,
                    'objective': 'binary',
                    'verbosity': -1,
                    'metric': 'binary', 
            })

    return hyperparameters_for_model


def models_hyperparameter_knn(company):

    n_neighbors_list = []
    
    if company == 'apple'or company == 'netflix':
        n_neighbors_list = [3, 50]
    elif company == 'amazon':
        n_neighbors_list = [50]
    elif company == 'meta':
        n_neighbors_list = [9]
    elif company == 'google':
        n_neighbors_list = [3, 5]
    
    hyperparameters_for_model = []
    
    for n_neighbors in n_neighbors_list:
        hyperparameters_for_model.append({
                        'n_neighbors': n_neighbors,
                        'n_jobs': -1
            })

    return hyperparameters_for_model


def models_hyperparameter_log(company):

    n_estimators_list = [10]
    
    hyperparameters_for_model = []
    
    for n_estimators in n_estimators_list:
        hyperparameters_for_model.append({
                        'max_iter': n_estimators,
                        'n_jobs': -1,
                        'random_state': 2,
                        'solver': 'liblinear'
            })

    return hyperparameters_for_model

In [16]:
def define_parameters(company, period, train_end, test_start, test_end, train_features_set, forecast_steps, models_dict, problem):
    
    list_of_configs = []
    
    model = None
    
#     for duration in [90, 120, 180, 240]:
    for train_start in [datetime(2015, 1, 1), datetime(2020, 1, 1), datetime(2023, 1, 1)]:
        for md in models_dict.values():
            if md == 'random_forest':
                hyperparameters_for_model = models_hyperparameter_random_forest(company)
            elif md == 'xgboost':
                hyperparameters_for_model = models_hyperparameter_xgboost(company)
            elif md == 'lightgbm':
                hyperparameters_for_model = models_hyperparameter_lgbm(company)
            elif md == 'knear_neighbors':
                hyperparameters_for_model = models_hyperparameter_knn(company)
            elif md == 'logistic_regression':
                hyperparameters_for_model = models_hyperparameter_log(company)
            else:
                print('Unknown model')
                return

            for hp in hyperparameters_for_model:

                if md == 'random_forest':
                    model = RandomForestClassifier(**hp)
                elif md == 'xgboost':
                    model = XGBClassifier(**hp)
                elif md == 'lightgbm':
                    model = LGBMClassifier(**hp)
                elif md == 'knear_neighbors':
                    model = KNeighborsClassifier(**hp)
                elif md == 'logistic_regression':
                    model = LogisticRegression(**hp)
                else:
                    print('Unknown model')

                for train_features in train_features_set:
    #                 'volume_lag_1', 'close_max_3_days', 'month'

#                     if company == 'apple' or company == 'google':
#                         if 'volume_lag_1' not in train_features and 'month' not in train_features:
#                             continue
#                     elif company == 'amazon':
#                         if 'volume_lag_1' not in train_features and 'close_max_3_days' not in train_features:
#                             continue                    
#     #                 elif company == 'meta':
#     #                     pass 
#     #                 if not ('volume_lag_1' in train_features or 'month' in train_features): continue                    
#                     elif company == 'netflix':
#                         if 'close_max_3_days' not in train_features and 'month' not in train_features:
#                             continue

                    config = {
                        'unique_uuid': str(uuid.uuid1()),
                        'train_start': train_start,
                        'train_end': train_end,
                        'test_start': test_start,
                        'test_end': test_end,
#                         'duration_training_history': duration,
                        'target_column': 'growth',
                        'train_features': train_features,
                        'path_to_result': f'/diploma_info/datalake/',
                        'forecast_periods': forecast_steps,
        #                 'hour_mean_value': 5,
                        'model_name': md,
                        'model': model,
                        'forecast_frequency': period,
                        'company': company,
                        'model_hyperparameters': hp,
                        'problem': problem
                    }

                    list_of_configs.append(config.copy())
    
    return list_of_configs

#### functions used in wfv service

In [17]:
def data(day, X_full_set, y_full_set, train_start, config, forecast_steps):

    X_train = X_full_set.loc[train_start:config["train_end"]]
    X_test = X_full_set.loc[config["test_start"]+timedelta(days=day):
                    config["test_start"]+timedelta(days=day+forecast_steps)]
    y_train = y_full_set.loc[train_start:config["train_end"]]
    y_test = y_full_set.loc[config["test_start"]+timedelta(days=day):
                    config["test_start"]+timedelta(days=day+forecast_steps)]
    
    return X_train, X_test, y_train, y_test

In [18]:
def standardize_mean_values(day, df_test, df_train, full_set, config):
    
    agg_cols = [col for col in config['train_features'] if col.endswith('_lag_1')] + \
               ['diff_open_value', 'open-prev_close', 'diff_close_value', 'growth_open'] + \
               [col for col in config['train_features'] if "close_m" in col]
    
    for agg in agg_cols:
        if agg in df_test.columns:
            try:
                num = df_test.loc[config["test_start"]+timedelta(days=day), agg]

            except KeyError as e:
                num = df_train[agg].iloc[-1]

            finally:

                _df = df_test.loc[config["test_start"]+timedelta(days=day):, agg]
                _df = _df.replace(_df.values, num)

#                 print(df_test.loc[config["test_start"]+timedelta(days=day):, agg], _df.values.ravel())

                df_test.loc[config["test_start"]:, agg] = _df.values.ravel()
    
    if 'open' in config['train_features']:
        
        idx_1 = df_test.iloc[1].name
        df_test.loc[idx_1, 'open'] = full_set.loc[idx_1, 'new_open']
        
        for n in range(df_test.loc[idx_1+timedelta(days=1):].shape[0]):
            
            _date = df_test.loc[idx_1+timedelta(days=1):].iloc[n].name
            df_test.loc[_date, 'open'] = full_set.loc[df_test.iloc[1+n].name, 'open'] + full_set.loc[idx_1-timedelta(days=5):idx_1, 'diff_open_value_mean_3_days'].mean()
        
    
    
    return df_test

In [19]:
def add_predictions(day, model_name, df_preds, y_pred_df, y_test, config):
    
    dates = y_test.index
    
    for date in dates:
        step_day = int((date-(config["test_start"]+timedelta(days=day))).days)
        df_preds.loc[date.strftime("%Y-%m-%d"), f'd-{step_day}'] = y_pred_df.loc[date.strftime("%Y-%m-%d"), 0]
    
    return df_preds

In [20]:
def estimations(day, df_stats, y_pred_df, y_test, config):
    
    dates = y_test.index
    
    for date in dates:
        step_day = int((date-(config["test_start"]+timedelta(days=day))).days)

        try:
            pred = y_pred_df.loc[date].values[0]
            real = y_test.loc[date].values[0]

            err = pred-real

            df_stats.loc[date, f'd-{step_day}' + '_is_true'] = 1 if (err == 0) else 0
            
        except ZeroDivisionError as e:
            print(e)

            df_stats.loc[date, f'd-{step_day}' + '_is_true'] = 0

        except KeyError as e:
            print(e)

            df_stats.loc[date, f'd-{step_day}' + '_is_true'] = 0
    
    return df_stats

In [21]:
def write_predictions(forecast_steps, df_preds, config, research_task_uuid, problem):
    
    for step in range(forecast_steps+1):
        try:
            pred = df_preds.loc[:, [f'd-{step}']].dropna().sort_index()
            pred.index.name = 'date_time'

            path_to_files = os.path.join(config['path_to_result'], "forecast", problem,
                                         config['company'], config['forecast_frequency'], config['model_name'], 
                                         f"research_task_{research_task_uuid}", 
                                         f"{config['model_name']}_{config['unique_uuid']}")
            if not os.path.isdir(path_to_files):
                os.makedirs(path_to_files)
                
            file_name = os.path.join(path_to_files, 
                    f"forecast_d-{step}_{config['model_name']}.csv")

            pd.DataFrame(pred).to_csv(file_name)

        except KeyError:
            pass

### wfv service

#### *add train_features grid (day_means, date_features, lag_features etc.)

#### *add checking LightGBM, Random_Forest, Linear_Classifier etc. 

#### *add lag_features (? check whats better: those or "mean"s) 

#### *am I able to add some factors like inflation/company's profit etc.

In [22]:
def run_wfv(full_set: pd.DataFrame, config: dict, research_task_uuid: str, forecast_steps: int, company: str, models_dict: dict, problem: str):
    
    X_full_set = full_set.loc[:, config['train_features']]
    y_full_set = full_set.loc[:, [config['target_column']]]
    
    if X_full_set.shape[0] != y_full_set.shape[0]:
        common_index = list(set(X_full_set.index) & set(y_full_set.index))
        common_index.sort()
        X_full_set = X_full_set.loc[common_index, :]
        y_full_set = y_full_set.loc[common_index, :]
    print(X_full_set.shape, y_full_set.shape)
    

    df_preds = pd.DataFrame()
    df_stats = pd.DataFrame()

    count_days = (test_end - test_start).days + 1
    
    
    model_name = config['model_name']
    print(model_name)
    
    model = config['model']

    unique_uuid = config['unique_uuid']
    
    if not os.path.isdir(config['path_to_result']):
        os.makedirs(config['path_to_result'])

    path_folder_result = os.path.join(config['path_to_result'], "wf_result", problem, config['company'], 
                                      config['forecast_frequency'], model_name,
                                      f"research_task_{research_task_uuid}")
    if not os.path.isdir(path_folder_result):
        os.makedirs(path_folder_result)
        
        

    for day in tqdm(range(count_days)):
        
        train_start = config.get('train_start', None)
        if train_start is None:
            if config.get('duration_training_history', None) is None:
                train_start = X_full_set.index[0]
                config['train_start'] = datetime(train_start.year, train_start.month, train_start.day)
            else:
                train_start = config['train_end'] + timedelta(days=day - config['duration_training_history'])

        try:

            X_train, X_test, y_train, y_test = data(day, X_full_set, y_full_set, train_start, config, forecast_steps)
    #         print(X_train, y_train)
            X_test = standardize_mean_values(day, X_test.copy(), X_train, full_set, config)
    #             print(test_start+timedelta(days=day), 'x_test: ', X_test.head(2))

            y_pred = model.fit(X_train, y_train).predict(X_test)

    #             print(y_pred)

            y_pred_df = pd.DataFrame(y_pred, index=y_test.index)
            df_preds = add_predictions(day, model_name, df_preds, y_pred_df, y_test, config)

            df_stats = estimations(day, df_stats, y_pred_df, y_test, config)
        


        except KeyError as e:
            print(e)
            continue
            
        except ValueError as e:
            print(e)
            continue


    write_predictions(forecast_steps, df_preds, config, research_task_uuid, problem)


    last_index = df_stats.index[-1]
    df_stats.loc[last_index, 'model_hyperparameters'] = str(config['model_hyperparameters'])
    df_stats.loc[last_index, 'train_features'] = str(config['train_features'])
    
    path_to_save_result_csv = os.path.join(path_folder_result, f'{model_name}_{unique_uuid}.csv')
    df_stats.round(2).to_csv(path_to_save_result_csv, date_format='%Y-%m-%d %H:%M:%S')
    
    config_to_save = config.copy()
    config_to_save.pop('model', None)
    with open(os.path.join(path_folder_result, f'{model_name}_{unique_uuid}.yaml'), 'w') as outfile:
        dump(config_to_save, outfile, default_flow_style=False)

In [23]:
# companies = ["NETFLIX"]
companies = ["AMAZON", "APPLE", "GOOGLE", "META", "NETFLIX"]
time_period = ["daily"]
# time_period = ["daily", "weekly", "monthly"]

problem = 'classification'
models_list = ['XGBoost', 'LightGBM', 'Random_Forest', 'KNear_Neighbors']
# models_list = ['XGBoost', 'LightGBM', 'Random_Forest', 'KNear_Neighbors', 'LOGistic_REGression']
# models_list = ['Prophet']
models_dict = dict([("".join(re.findall('([A-Z])', k)).lower(), k.lower()) for k in models_list])

forecast_steps = 4        # means that forecast will be made on {n} futute periods 
 

for company, period in list(itertools.product(companies, time_period)):
    
    full_set, train_end, test_start, test_end, train_features_set = init(company, period)
    print(f'company: {company}\t period: {period}\t train ends: {train_end}\t test starts: {test_start}')
    
    _research_task_uuid = str(uuid.uuid1())
    print(f'_research_task_uuid = {_research_task_uuid}\n')
    
    configs = define_parameters(company.lower(), period.lower(), train_end, test_start, test_end, train_features_set, forecast_steps, models_dict, problem)
    print(f'count_configs {len(configs)} \n')
    
    for _ in configs:
        print(_['model_name'], '==', _, '\n')
        
        run_wfv(full_set, _, _research_task_uuid, forecast_steps, company, models_dict, problem)

company: AMAZON	 period: daily	 train ends: 2023-12-29 00:00:00	 test starts: 2024-01-02 00:00:00
_research_task_uuid = cae1a9b5-0e08-11ef-aaec-c0e434d84b22

count_configs 36 

xgboost == {'unique_uuid': 'cae1a9b6-0e08-11ef-a086-c0e434d84b22', 'train_start': datetime.datetime(2015, 1, 1, 0, 0), 'train_end': datetime.datetime(2023, 12, 29, 0, 0), 'test_start': datetime.datetime(2024, 1, 2, 0, 0), 'test_end': datetime.datetime(2024, 2, 8, 0, 0), 'target_column': 'growth', 'train_features': ['open', 'volume_lag_1', 'usa_inflation_%'], 'path_to_result': '/diploma_info/datalake/', 'forecast_periods': 4, 'model_name': 'xgboost', 'model': XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eta=0.3, eval_metric='rmse', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
    

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:02<00:00, 17.39it/s]


xgboost == {'unique_uuid': 'cae1d0c2-0e08-11ef-904f-c0e434d84b22', 'train_start': datetime.datetime(2015, 1, 1, 0, 0), 'train_end': datetime.datetime(2023, 12, 29, 0, 0), 'test_start': datetime.datetime(2024, 1, 2, 0, 0), 'test_end': datetime.datetime(2024, 2, 8, 0, 0), 'target_column': 'growth', 'train_features': ['open', 'volume_lag_1', 'diff_close_value'], 'path_to_result': '/diploma_info/datalake/', 'forecast_periods': 4, 'model_name': 'xgboost', 'model': XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eta=0.3, eval_metric='rmse', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_d

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 51.58it/s]


xgboost == {'unique_uuid': 'cae1d0c3-0e08-11ef-b508-c0e434d84b22', 'train_start': datetime.datetime(2015, 1, 1, 0, 0), 'train_end': datetime.datetime(2023, 12, 29, 0, 0), 'test_start': datetime.datetime(2024, 1, 2, 0, 0), 'test_end': datetime.datetime(2024, 2, 8, 0, 0), 'target_column': 'growth', 'train_features': ['open', 'volume_lag_1', 'growth_open'], 'path_to_result': '/diploma_info/datalake/', 'forecast_periods': 4, 'model_name': 'xgboost', 'model': XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eta=0.3, eval_metric='rmse', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=

...

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:01<00:00, 36.03it/s]


knear_neighbors == {'unique_uuid': '98a48d6f-0e14-11ef-8e3e-c0e434d84b22', 'train_start': datetime.datetime(2023, 1, 1, 0, 0), 'train_end': datetime.datetime(2023, 12, 29, 0, 0), 'test_start': datetime.datetime(2024, 1, 2, 0, 0), 'test_end': datetime.datetime(2024, 2, 8, 0, 0), 'target_column': 'growth', 'train_features': ['open', 'volume_lag_1', 'diff_close_value'], 'path_to_result': '/diploma_info/datalake/', 'forecast_periods': 4, 'model_name': 'knear_neighbors', 'model': KNeighborsClassifier(n_jobs=-1, n_neighbors=50), 'forecast_frequency': 'daily', 'company': 'netflix', 'model_hyperparameters': {'n_neighbors': 50, 'n_jobs': -1}, 'problem': 'classification'} 

(2346, 3) (2346, 1)
knear_neighbors


100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:01<00:00, 35.84it/s]


knear_neighbors == {'unique_uuid': '98a48d70-0e14-11ef-9436-c0e434d84b22', 'train_start': datetime.datetime(2023, 1, 1, 0, 0), 'train_end': datetime.datetime(2023, 12, 29, 0, 0), 'test_start': datetime.datetime(2024, 1, 2, 0, 0), 'test_end': datetime.datetime(2024, 2, 8, 0, 0), 'target_column': 'growth', 'train_features': ['open', 'volume_lag_1', 'growth_open'], 'path_to_result': '/diploma_info/datalake/', 'forecast_periods': 4, 'model_name': 'knear_neighbors', 'model': KNeighborsClassifier(n_jobs=-1, n_neighbors=50), 'forecast_frequency': 'daily', 'company': 'netflix', 'model_hyperparameters': {'n_neighbors': 50, 'n_jobs': -1}, 'problem': 'classification'} 

(2346, 3) (2346, 1)
knear_neighbors


100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:01<00:00, 34.91it/s]


## View results

In [24]:
# df_preds['true_val'] = df.growth
# df_preds 

In [25]:
# df_preds[(df_preds['true_val'] - df_preds['xgb_10_3_d-1'])==0]

# df_errors = pd.DataFrame()


# for d in [0, 1, 2, 3]:
#     for n in [k for k in df_preds.columns if f'd-{d}' in k]:
#         acc = round(abs(df_preds[(df_preds['true_val'] - df_preds[n])==0].shape[0]/df_preds.shape[0] - 1), 2) * 100
        
# #         print(n, acc)
#         df_errors.loc[d, n.split("_d")[0]] = acc
        
#     print(df_errors.iloc[:, :2])

In [26]:
# df_errors[[k for k in df_errors.columns if 'lin' in k]]

In [27]:
full_set

,open,high,low,close,adj_close,volume,diff_close_value,diff_open_value,open-prev_close,close-open_lag_1,...,close_max_7_days,close_min_7_days,close_mean_7_days,1_USD_to_EUR_lag_1,1_EUR_to_USD_lag_1,1_UAH_to_USD_lag_1,1_USD_to_UAH_lag_1,1_EUR_to_UAH_lag_1,1_UAH_to_EUR_lag_1,usa_inflation_%
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,49.151428,50.331429,48.731430,49.848572,49.848572,13475000.0,1.047142,0.118572,0.349998,-0.231426,...,49.0,48.0,49.0,0.8265,1.2098,0.06322,15.819,19.138,0.05225,0.76
2015-01-05,49.258572,49.258572,47.147144,47.311428,47.311428,18165000.0,-2.537144,0.107143,-0.590000,0.697144,...,50.0,48.0,49.0,0.8370,1.1946,0.06265,15.962,18.900,0.05243,0.76
2015-01-06,47.347141,47.639999,45.661430,46.501431,46.501431,16037700.0,-0.809998,-1.911430,0.035713,-1.947144,...,50.0,47.0,49.0,0.8391,1.1918,0.06322,15.818,18.852,0.05305,0.76
2015-01-07,47.347141,47.421429,46.271427,46.742859,46.742859,9849700.0,0.241428,0.000000,0.845711,-0.845711,...,50.0,47.0,48.0,0.8377,1.1937,0.06317,15.830,18.896,0.05292,0.76
2015-01-08,47.119999,47.835712,46.478573,47.779999,47.779999,9601900.0,1.037140,-0.227142,0.377140,-0.604282,...,50.0,47.0,48.0,0.8461,1.1819,0.06317,15.830,18.709,0.05345,0.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-23,556.489990,579.210022,555.280029,577.750000,577.750000,4975500.0,23.150024,5.830017,1.890015,3.940002,...,623.0,555.0,597.0,0.9386,1.0654,0.02513,39.795,42.396,0.02359,3.48
2024-04-24,574.309998,576.909973,551.299988,555.119995,555.119995,5355300.0,-22.630005,17.820007,-3.440002,21.260010,...,618.0,555.0,591.0,0.9344,1.0702,0.02529,39.540,42.314,0.02363,3.48
2024-04-25,549.460022,566.539978,545.710022,564.799988,564.799988,3807100.0,9.679993,-24.849976,-5.659973,-19.190002,...,618.0,555.0,583.0,0.9349,1.0697,0.02529,39.543,42.298,0.02364,3.48
